In [0]:
# File location and type
file_location = "/FileStore/tables/Superstore_Sales.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark\
    .read\
    .format(file_type)\
    .option("inferSchema", infer_schema)\
    .option("header", first_row_is_header)\
    .option("sep", delimiter)\
    .load(file_location)

display(df)

In [0]:
# Create a view or table
df.createOrReplaceTempView("Superstore_Sales_csv")

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `Superstore_Sales_csv`

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "Superstore_Sales_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
df1 = spark.read\
    .format("csv")\
    .option("header", True)\
    .option("inferSchema", True)\
    .option("path", "/FileStore/tables/Superstore_Sales.csv")\
    .load()

df1.orderBy("COUNTRY")\
   .groupBy("COUNTRY")\
   .agg(expr("SUM(QUANTITYORDERED) as Total_Quantity"),
        expr("SUM(SALES) as Total_Sales"))\
   .where(col("COUNTRY")=="Australia")\
   .show()


In [0]:
from pyspark.sql.functions import *

# Input data - List
myList = [(1,"2013-07-25",11599,"CLOSED"),
          (2,"2014-07-25",256,"PENDING_PAYMENT"),
          (3,"2013-07-25",11599,"COMPLETE"),
          (4,"2019-07-25",8827,"CLOSED")]

df1=spark.createDataFrame(myList)\
    .toDF("orderid","orderdate","customerid","status")

newDf = df1\
    .withColumn("date1",to_timestamp(col("orderdate")))\
    .withColumn("newid",monotonically_increasing_id())\
    .dropDuplicates(["orderdate","customerid"])\
    .sort("orderdate")

df1.show()
newDf.show()

In [0]:
from pyspark.sql.functions import *

# Input data - List
myList = [(1,"2013-07-25",11599,"CLOSED"),
          (2,"2014-07-25",256,"PENDING_PAYMENT"),
          (3,"2013-07-25",11599,"COMPLETE"),
          (4,"2019-07-25",8827,"CLOSED")]

spark.createDataFrame(myList).show()

df1=spark.createDataFrame(myList)\
    .toDF("orderid","orderdate","customerid","status").show()


In [0]:
from pyspark.sql.functions import *
# Sample data
myList = [("anusha",16,"Mysore"),
          ("narendra",22,"Hydrabad"),
          ("ankit",25,"Toronto"),
          ("suchet",14,"Bangalore")]

# Create dataframe from data source Mylist
df1= spark.createDataFrame(myList).toDF("name","age","city")

# ageCheck User defined Function is created.
def ageCheck(age):
    if(age > 18):
      return "Y"
    else:
      return "N"
# Function is registered with spark catalog
spark.udf.register("parseAgeFunction",ageCheck,StringType())

# New column Adult is created by using UDF and by passing age as parameter.
df2 = df1.withColumn("adult",expr("parseAgeFunction(age)"))
df1.show()
df2.show()

In [0]:
from pyspark.sql.functions import *

# Input data - List
myList = [(1,"2013-07-25",11599,"CLOSED"),
          (2,"2014-07-25",256,"PENDING_PAYMENT"),
          (3,"2013-07-25",11599,"COMPLETE"),
          (4,"2019-07-25",8827,"CLOSED")]

spark.createDataFrame(myList).show()

df1=spark.createDataFrame(myList)\
    .toDF("orderid","orderdate","customerid","status").show()



In [0]:
from pyspark.sql.functions import *

df1 = spark.read\
   .format("csv")\
   .option("inferSchema", True)\
   .option("path", "/FileStore/tables/sample_data.csv")\
   .load()

df2 = df1.toDF("name","age","city")

# ageCheck User defined Function (UDF) is created.
def ageCheck(age):
    if(age > 18):
      return "Y"
    else:
      return "N"

# Function is registered with spark catalog
spark.udf.register("parseAgeFunction",ageCheck,StringType())

# New column Adult is created by using UDF and age as the parameter.
df3 = df2.withColumn("adult",expr("parseAgeFunction(age)"))

# Print df3 result to console.
df1.show()
df2.show()
df3.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

df1 = spark.read\
   .format("csv")\
   .option("inferSchema", True)\
   .option("path", "/FileStore/tables/sample_data.csv")\
   .load()

df1.show()

df2 = df1.toDF("name","age","city")

# New column Adult is created by using UDF and age as the parameter.
df2.createOrReplaceTempView("sam_data")

df3 = spark.sql("""
SELECT 
name,
age,
city,
case when age < 18 then 'N' else 'Y' end as adult
FROM sam_data """)

# Print df3 result to console.
df3.show()

In [0]:
from pyspark.sql.functions import *

# Read the data from sample_data.csv and load it to dataframe.
df1 = spark.read\
   .format("csv")\
   .option("inferSchema", True)\
   .option("path", "/FileStore/tables/sample_data.csv")\
   .load()

df2 = df1.toDF("name","age","city")

# Create database lohith, if it is not existed.
spark.sql("create database if not exists lohith")

# Create Table sample1 under the database lohith, and overwrite the data.
df2.write\
    .format("csv")\
    .mode("overwrite")\
    .saveAsTable("lohith.sample")

# OVERWRITE the data to the DBFS location "/FileStore/tables/output"
# df2.write\
#     .format("csv")\
#     .mode("overwrite")\
#     .option("path","/FileStore/tables/output")

In [0]:
from pyspark.sql.functions import *

# Read the data from unstructured_data.txt and load it to dataframe.
df1 = spark.read.text("/FileStore/tables/unstructured_data.txt")

print("Input unstructured data")
df1.show(truncate = False) # Printing to the console

# Convert unstructued data to structured data using Regular expressions
regular_expn = r'^(\S+) (\S+)\t(\S+)\,(\S+)'

df2 = df1.select(regexp_extract('value',regular_expn,1).alias("No"),
                 regexp_extract('value',regular_expn,2).alias("name"),
                 regexp_extract('value',regular_expn,3).alias("company"),
                 regexp_extract('value',regular_expn,4).alias("department"))

print("output stuctured data")
df2.show() # Printing to the console

In [0]:
from pyspark.sql.functions import *

# Read the data from unstructured_data.txt and load it to dataframe.
df1 = spark.read.text("/FileStore/tables/unstructured_data1.txt")

print("Input unstructured data")
df1.show(truncate = False) # Printing to the console

# Convert unstructued data to structured data using Regular expressions
regular_expn = r'^(\S+)-(\S+)-(\S+)-(\S+)'

df2 = df1.select(regexp_extract('value',regular_expn,1).alias("No"),
                 regexp_extract('value',regular_expn,2).alias("name"),
                 regexp_extract('value',regular_expn,3).alias("company"),
                 regexp_extract('value',regular_expn,4).alias("department"))

print("output stuctured data")
df2.show() # Printing to the console